In [1]:
import pandas as pd
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setMaster('local').setAppName('Filtering')
sc = SparkContext(conf=conf)

In [21]:
col_names=['station_id', 'date', 'observation_type', 'temperature', 'a', 'b', 'c', 'd']
df = pd.read_csv('../datasets/1800.csv', names=col_names)
df.head()

,station_id,date,observation_type,temperature,a,b,c,d
0,ITE00100554,18000101,TMAX,-75,NaN,NaN,E,NaN
1,ITE00100554,18000101,TMIN,-148,NaN,NaN,E,NaN
2,GM000010962,18000101,PRCP,0,NaN,NaN,E,NaN
3,EZE00100082,18000101,TMAX,-86,NaN,NaN,E,NaN
4,EZE00100082,18000101,TMIN,-135,NaN,NaN,E,NaN


In [19]:
df[df['observation_type'] == 'TMIN'].groupby(by='station_id').min()

,date,observation_type,min_temp,a,c,d
station_id,,,,,,
EZE00100082,18000101,TMIN,-135,NaN,E,NaN
ITE00100554,18000101,TMIN,-148,NaN,E,NaN


In [22]:
df[df['observation_type'] == 'TMAX'].groupby(by='station_id').max()

,date,observation_type,temperature,a,c,d
station_id,,,,,,
EZE00100082,18001231,TMAX,323,NaN,E,NaN
ITE00100554,18001231,TMAX,323,NaN,E,NaN


In [33]:
# min and max temperatures
def parse_line(line):
    fields = line.split(',')
    return fields[0], fields[2], float(fields[3])

rdd = sc.textFile('../datasets/1800.csv')
parsed_lines = rdd.map(parse_line)
min_temps = parsed_lines.filter(lambda x: x[1] == 'TMIN') \
                        .map(lambda x: (x[0], x[2])) \
                        .reduceByKey(lambda x, y: min(x, y))

max_temps = parsed_lines.filter(lambda x: x[1] == 'TMAX') \
                       .map(lambda x: (x[0], x[2])) \
                       .reduceByKey(lambda x, y: max(x, y))

print(f'min temperatures: {min_temps.collect()}')
print(f'max temperatures: {max_temps.collect()}')

min temperatures: [('ITE00100554', -148.0), ('EZE00100082', -135.0)]
max temperatures: [('ITE00100554', 323.0), ('EZE00100082', 323.0)]
